In [1]:
# !pip install transformers
# !pip install vocab

In [2]:
import transformers
import argparse
from sklearn.metrics import accuracy_score


from config import *
from model import CLModel, FGM

from data_process import *
from util_methods import *
from spcl_loss import SupProtoConLoss

In [3]:
no_label_data = 6
data_path = f"../../DataPreprocess/Cleaned_Data/{no_label_data}_label/clean_data.csv"
train_data_path = f"../../DataPreprocess/Cleaned_Data/{no_label_data}_label/train_data.csv"
dev_data_path = f"../../DataPreprocess/Cleaned_Data/{no_label_data}_label/dev_data.csv"
test_data_path = f"../../DataPreprocess/Cleaned_Data/{no_label_data}_label/test_data.csv"

In [4]:
train_data = pd.read_csv(train_data_path).fillna("")
valid_data = pd.read_csv(dev_data_path).fillna("")
test_data = pd.read_csv(test_data_path).fillna("")
df = pd.read_csv(data_path).fillna("")

print(f"Train Set Shape: {train_data.shape}")
print(f"Valid Set Shape: {valid_data.shape}")
print(f"Test Set Shape: {test_data.shape}")
print(f"Full Set Shape: {df.shape}")

df.head(5)

Train Set Shape: (10888, 12)
Valid Set Shape: (3077, 12)
Test Set Shape: (1568, 12)
Full Set Shape: (15533, 12)


,Index,Utterance,Speaker,Id_speaker,Utterance_id,Date,Time,Emotion,Emotion_Mutiple,Dialog_id,Label,Utterance_clean
0,1,Bao tiền,Nguyễn Thanh Tú,100031059109987,1,18/02/2022,08:07:47,Neutral,Neutral,1,0,Bao tiền
1,2,Nguyễn Thanh Tú bạn có khum haha,Nguyễn Thị Diễm,100007602498241,2,18/02/2022,08:08:10,Joy,Joy,1,1,bạn có khum haha
2,3,Nguyễn Thị Diễm nổ giá đii đừng ib.,Nguyễn Thanh Tú,100031059109987,3,18/02/2022,08:08:27,Anger,Anger,1,2,nổ giá đii đừng ib .
3,4,T có nha,Dao Phuong Anh,100009157681703,1,18/02/2022,08:37:06,Neutral,Neutral,2,0,T có nha
4,5,Dao Phuong Anh check ib ạ,Nguyễn Thị Diễm,100007602498241,2,18/02/2022,08:37:18,Neutral,Neutral,2,0,check ib ạ


In [5]:
# Trả về tham số hoặc tham số warmup trong model theo từng thành phần trong model
def get_paramsgroup(model, warmup=False):
    no_decay = ['bias', 'LayerNorm.weight'] # no_regularization
    pre_train_lr = CONFIG['ptmlr'] # pre-train model learning rate

    bert_params = list(map(id, model.f_context_encoder.parameters())) # lấy tham số từ model encoder (SimCSE)
    params = []
    warmup_params = []
    for name, param in model.named_parameters(): # Duyệt qua các parameter
        lr = CONFIG['lr'] # Learning rate
        weight_decay = 0.01 # Regularization weight
        if id(param) in bert_params: # Nếu như đang xét là pre_train model thì dùng pre_train learning rate
            lr = pre_train_lr
        if any(nd in name for nd in no_decay): # Kiểm tra nếu
            weight_decay = 0
        params.append({
            'params': param,
            'lr': lr,
            'weight_decay': weight_decay
        })
        warmup_params.append({
            'params':
            param,
            'lr':
            CONFIG['ptmlr'] / 4 if id(param) in bert_params else lr,
            'weight_decay':
            weight_decay
        })
    if warmup:
        return warmup_params
    params = sorted(params, key=lambda x: x['lr'])
    return params

In [6]:
# Train model
def train_epoch(model,
                optimizer,
                lr_scheduler,
                trainset,
                centers,
                centers_mask=None,
                epoch=0,
                max_step=-1,
                train_obj='all'):
    # Train
    model.train()
    inner_model = model.module if hasattr(model, 'module') else model # Lấy nếu module CL lồng trong module (FGM()) nếu có
    ce_loss_func = torch.nn.CrossEntropyLoss(ignore_index=-1) # Cross Entropy Loss
    # SPCL loss
    spcl_loss = SupProtoConLoss(
        num_classes=no_label_data,
        temp=CONFIG['temperature'],
        pool_size=CONFIG['pool_size'],
        support_set_size=CONFIG['support_set_size'],
        centers=centers)
    # Tích lũy 1 số lượng batch_size nhất định rồi update thay vì update theo từng batch
    accumulation_steps = CONFIG['accumulation_steps']
    # Lấy tập mẫu từ dataset (DistributedSampler dùng cho trường hợp dùng hệ phân tán)
    sampler = RandomSampler(
        trainset) if CONFIG['local_rank'] == -1 else DistributedSampler(
            trainset)
    if CONFIG['local_rank'] != -1:
        sampler.set_epoch(epoch)
    # Dataloader - (drop_last = True -> bỏ đi batch cuối cùng nếu như không đủ batch_size)
    dataloader = DataLoader(trainset,
                            batch_size=CONFIG['batch_size'],
                            sampler=sampler,
                            num_workers=0,
                            drop_last=True)
    # Hiện thị tiến trình huấn luyện
    tq_train = tqdm(total=max_step if max_step > 0 else len(dataloader),
                    position=0,leave=True,
                    disable=CONFIG['local_rank'] not in [-1, 0])
    
    # Tính toán loss
    def calc_loss(sentences, emotion_idxs, labels):
        ccl_reps = inner_model.gen_f_reps(sentences) # Encode
        
        # Loại direct loss (tính theo cross entropy)
        if train_obj == 'ce':
            direct_loss = ce_loss_func(inner_model.predictor(ccl_reps),
                                    emotion_idxs)
        else:
            direct_loss = ce_loss_func(inner_model.predictor(ccl_reps.detach()),
                                   emotion_idxs)
            
        # ins_cl_loss tính theo spcl hoặc spdcl
        ins_cl_loss = torch.zeros(1).to(CONFIG['device'])
        if train_obj == 'spcl':
            ins_cl_loss = spcl_loss(ccl_reps, labels)
        if train_obj == 'spdcl':
            ins_cl_loss = spcl_loss(ccl_reps, labels, decoupled=True)
        
        # Loss tổng thể tính theo direct loss + ins_cl_loss(=0 nếu train_obj = "CE")
        loss = direct_loss + ins_cl_loss
        
        # Đặt thông báo huấn luyện trả về mất mát
        tq_train.set_description(
            'direct loss {:.2f}, instance_cl_loss {:.2f}'
            .format(direct_loss.item(),
                    ins_cl_loss.item() if train_obj in ['spcl', 'spdcl'] else 0))
        return loss
    
    # FGM model
    fgm = FGM(inner_model)
    
    # Duyệt qua dataloader
    for batch_id, batch_data in enumerate(dataloader):
        batch_data = [x.to(inner_model.device()) for x in batch_data]
        
        sentences = batch_data[0]

        emotion_idxs = batch_data[1].reshape(-1)
        labels = batch_data[2].reshape(-1)
        
        # Tính loss + back propagation
        loss = calc_loss(sentences, emotion_idxs, labels)
        loss = loss / accumulation_steps
        loss.backward()
        
        # Nếu sử dụng FGM
        if CONFIG['fgm']:
            fgm.attack()
            loss = calc_loss(sentences, emotion_idxs, labels)
            loss = loss / accumulation_steps
            loss.backward()
            fgm.restore()
        
        # Clip gradient để tránh overfit
        nn.utils.clip_grad_norm_(inner_model.parameters(), max_norm=5, norm_type=2)
        tq_train.update()
        
        # Cập nhật theo accumulation_step nhất định
        if batch_id % accumulation_steps == 0:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
        if max_step > 0 and batch_id > max_step:
            optimizer.zero_grad()
            break
    optimizer.zero_grad()
    tq_train.close()

In [7]:
# Train model
def train(model, train_dialogues, dev_dialogues, test_dialogues):
    devset = build_dataset(dev_dialogues)
    testset = build_dataset(test_dialogues)

    tq_epoch = tqdm(total=CONFIG['epochs'],
                    position=0,leave=True,
                    disable=CONFIG['local_rank'] not in [-1, 0])
    centers = None

    if CONFIG['local_rank'] != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model,
            device_ids=[CONFIG['local_rank']],
            output_device=CONFIG['local_rank'],
            find_unused_parameters=True)
    optimizer = torch.optim.AdamW(
        get_paramsgroup(
            model.module if hasattr(model, 'module') else model))
    checkpoint_saver = FGM(model)
    # train
    best_f1 = -1
    epochs_not_update = 0
    train_obj = CONFIG['train_obj']
    for epoch in range(CONFIG['epochs']+1):
        tq_epoch.set_description('training on epoch {}'.format(epoch))
        tq_epoch.update()

        trainset = build_dataset(train_dialogues, train=True)

        if CONFIG['local_rank'] in [-1, 0]:
            all_reps, all_corr_labels = gen_all_reps(model, trainset)

            logging.info('clustering...')
            centers, centers_mask, cluster2dataid, cluster2classid, all_centers = cluster(
                all_reps, all_corr_labels, init_centers=centers, epoch=epoch)
            num_data = all_reps.shape[0]
            if epoch > 0:
                f1, acc_diriect, acc_cluster = test(model,
                          devset,
                          centers,
                          centers_mask,
                          desc='dev @ epoch {}'.format(epoch - 1))
                if f1 > best_f1:
                    epochs_not_update = 0
                    os.system('rm {}/models/*'.format(CONFIG['temp_path']))
                    os.system('rm {}/centers/*'.format(CONFIG['temp_path']))

                    best_f1 = f1
                    model_to_save = model.module if hasattr(
                        model, "module") else model
                    torch.save(
                        model_to_save, CONFIG['temp_path'] +
                        '/models/f1_{:.4f}_@epoch{}.pkl'.format(
                            best_f1, epoch - 1))
                    torch.save(
                        centers, CONFIG['temp_path'] +
                        '/centers/f1_{:.4f}_@epoch{}.pkl'.format(
                            best_f1, epoch - 1))
                    torch.save(
                        centers_mask, CONFIG['temp_path'] +
                        '/centers/f1_{:.4f}_@epoch{}.msk'.format(
                            best_f1, epoch - 1))

                    f1, acc_diriect, acc_cluster = test(model,
                              testset,
                              centers,
                              centers_mask,
                              desc='new best test @ epoch {}'.format(
                                  epoch - 1))
                    # checkpoint_saver.save_checkpoint()
                else:
                    epochs_not_update += 1
                    # checkpoint_saver.load_checkpoint()
                    torch.cuda.empty_cache()
                if epochs_not_update >= 5:
                    break

            selection, cluster_idxs = gen_cl_data(all_reps,
                                                  all_centers,
                                                  cluster2dataid,
                                                  cluster2classid,
                                                  epoch=epoch)
            # st = 1 - epoch / 10
            # ed = epoch / 10
            st = 1 - epoch / CONFIG['epochs']
            ed = epoch / CONFIG['epochs']
            num_data = len(selection)
            selection = torch.LongTensor(selection)
            prob_list = [
                st + (ed - st) / (num_data - 1) * i for i in range(num_data)
            ]
            prob_tensor = torch.FloatTensor(prob_list)
            rand_prob_tensor = torch.bernoulli(torch.ones(num_data) * 0.5)
            if CONFIG['cl']:
                sample = torch.bernoulli(prob_tensor).long()
            else:
                sample = torch.bernoulli(rand_prob_tensor).long()
            selection = selection * sample
            sample_results = selection[torch.nonzero(selection)]
            all_idxs = sample_results.numpy().tolist()

            epoch_trainset = TensorDataset(trainset[all_idxs][0],
                                           trainset[all_idxs][1],
                                           cluster_idxs[all_idxs])
            torch.save(epoch_trainset,
                       CONFIG['temp_path'] + '/temp/train_set.pkl')
            torch.save(centers, CONFIG['temp_path'] + '/temp/centers.pkl')
            torch.save(centers_mask,
                       CONFIG['temp_path'] + '/temp/centers_mask.pkl')
        if CONFIG['local_rank'] != -1:
            torch.distributed.barrier()

        epoch_trainset = torch.load(CONFIG['temp_path'] +
                                    '/temp/train_set.pkl',
                                    map_location=CONFIG['device'])
        centers = torch.load(CONFIG['temp_path'] + '/temp/centers.pkl',
                             map_location=CONFIG['device'])
        centers_mask = torch.load(CONFIG['temp_path'] +
                                  '/temp/centers_mask.pkl',
                                  map_location=CONFIG['device'])
        num_training_steps = len(epoch_trainset)//(CONFIG['batch_size'] * CONFIG['accumulation_steps'])
        num_warmup_steps = min(CONFIG['warm_up'], num_training_steps) if epoch == 0 else 0
        lr_scheduler = transformers.get_cosine_schedule_with_warmup(
                                    optimizer,
                                    num_warmup_steps=num_warmup_steps,
                                    num_training_steps=num_training_steps
                                    )
        if epoch < CONFIG['epochs']:
            train_epoch(model,
                    optimizer,
                    lr_scheduler,
                    epoch_trainset,
                    centers,
                    centers_mask,
                    epoch,
                    train_obj=train_obj)

    tq_epoch.close()
    if CONFIG['local_rank'] in [0, -1]:
        model, centers, centers_mask = load_latest()
        f1, acc_diriect, acc_cluster = test(model, testset, centers, centers_mask)
        print('best f1 test is {:.4f}'.format(f1), flush=True)
        print('acc direct test  is {:.4f}'.format(acc_diriect), flush=True)
        print('acc cluster test is {:.4f}'.format(acc_cluster), flush=True)




In [8]:
def test(model, data, centers, centers_mask, desc=''):

    model.eval()
    inner_model = model.module if hasattr(model, 'module') else model
    y_true_list = []
    direct_list = [[], [], [], []]
    cluster_list = [[], [], [], []]
    sampler = SequentialSampler(data)
    dataloader = DataLoader(
        data,
        batch_size=CONFIG['batch_size'],
        sampler=sampler,
        num_workers=0,  # multiprocessing.cpu_count()
    )
    tq_test = tqdm(total=len(dataloader), desc="testing", position=0,leave=True)
    for batch_id, batch_data in enumerate(dataloader):
        batch_data = [x.to(inner_model.device()) for x in batch_data]
        sentences = batch_data[0]
        emotion_idxs = batch_data[1].reshape(-1)
        with torch.no_grad():
            ccl_reps = inner_model.gen_f_reps(sentences)
        cluster_outputs, direct_outputs = [], []

        feature_list = [ccl_reps]
        num_feature = len(feature_list)

        for idx, feature in enumerate(feature_list):
            #
            outputs = inner_model(feature, centers, score_func)
            outputs -= (1 - centers_mask) * 2
            cluster_outputs.append(torch.argmax(outputs.max(-1)[0], -1))
            direct_outputs.append(
                torch.argmax(inner_model.predictor(feature), -1))
        for batch_id in range(emotion_idxs.shape[0]):
            if emotion_idxs[batch_id] > -1:
                for idx in range(num_feature):
                    direct_list[idx].append(
                        direct_outputs[idx][batch_id].item())
                    cluster_list[idx].append(
                        cluster_outputs[idx][batch_id].item())
                y_true_list.append(emotion_idxs[batch_id].item())
        tq_test.update()
    direct_f1_scores = [
        f1_score(y_true=y_true_list,
                 y_pred=direct_list[idx],
                 average='weighted') for idx in range(num_feature)
    ]
    cluster_f1_scores = [
        f1_score(y_true=y_true_list,
                 y_pred=cluster_list[idx],
                 average='weighted') for idx in range(num_feature)
    ]
    
    # Calculate accuracy
    direct_acc = [accuracy_score(y_true=y_true_list,
                                  y_pred=direct_list[idx]) for idx in range(num_feature)]
    cluster_acc = [accuracy_score(y_true=y_true_list,
                                   y_pred=cluster_list[idx]) for idx in range(num_feature)]

    # f1 = max(max(direct_f1_scores), max(cluster_f1_scores))
    f1 = max(cluster_f1_scores)
    clus_acc =max(cluster_acc)
    dir_acc = max(direct_acc)
    print('\n{} best w-f1 is {:.4f}'.format(desc, f1), flush=True)
    print('direct f1 cls {:.4f}'.format(*direct_f1_scores))
    print('cluster f1 cls {:.4f}'.format(*cluster_f1_scores),
          flush=True)

    return f1, dir_acc, clus_acc


In [9]:
def load_latest():
    model_path = CONFIG['temp_path'] + '/models'
    lst = os.listdir(model_path)
    lst = list(filter(lambda item: item.endswith('.pkl'), lst))
    lst.sort(key=lambda x: os.path.getmtime(os.path.join(model_path, x)))
    model = torch.load(os.path.join(model_path, lst[-1]))
    logging.info(
        'model checkpoint {} is loaded'.format(
            os.path.join(model_path, lst[-1])), )
    center_path = CONFIG['temp_path'] + '/centers'
    lst = os.listdir(center_path)
    lst = list(filter(lambda item: item.endswith('.pkl'), lst))
    lst.sort(key=lambda x: os.path.getmtime(os.path.join(center_path, x)))
    centers = torch.load(os.path.join(center_path, lst[-1]))
    logging.info(
        'center checkpoint {} is loaded'.format(
            os.path.join(center_path, lst[-1])), )

    lst = os.listdir(center_path)
    lst = list(filter(lambda item: item.endswith('.msk'), lst))
    lst.sort(key=lambda x: os.path.getmtime(os.path.join(center_path, x)))
    centers_mask = torch.load(os.path.join(center_path, lst[-1]))
    logging.info(
        'centers mask checkpoint {} is loaded'.format(
            os.path.join(center_path, lst[-1])), )

    return model, centers, centers_mask

In [10]:
class Arguments:
    def __init__(self, pool_size = 256, support_set_size=64, temperature = 0.05):
        self.test = True
        self.train = True
        self.finetune = False
        self.cl = True
        self.print_error = True
        self.output_mlp = False
        self.fgm = False
        self.batch_size = CONFIG['batch_size']
        self.seed = 2333 
        self.pool_size = pool_size #CONFIG['pool_size']
        self.support_set_size = support_set_size #CONFIG['support_set_size']
        self.epochs = 16 #CONFIG['epochs']
        self.avg_cluster_size = CONFIG['avg_cluster_size']
        self.lr = CONFIG['lr']
        self.ptmlr = CONFIG['ptmlr']
        self.task_name = 'ERCVN'
        self.fp_16 = False
        self.warm_up = CONFIG['warm_up']
        self.dropout = CONFIG['dropout']
        self.temperature = temperature #CONFIG['temperature']
        self.bert_path = "SIMCSE_pretrained" #CONFIG['bert_path']
        self.train_obj = 'spcl' #CONFIG['train_obj']
        self.data_path = CONFIG['data_path']
        self.temp_path = CONFIG['temp_path']
        self.accumulation_steps = CONFIG['accumulation_steps']
        self.local_rank = -1
        self.n_gpu = 0
        self.no_cuda = False
        self.device = 'cuda:0'

In [11]:
if __name__ == '__main__':
    args = Arguments(256, 64, 0.05)

    if args.local_rank == -1:
        device = torch.device("cuda" if torch.cuda.is_available()
                              and not args.no_cuda else "cpu")
        args.n_gpu = 0 if args.no_cuda else torch.cuda.device_count()
    else:
        torch.cuda.set_device(args.local_rank)
        device = torch.device("cuda", args.local_rank)
        os.environ["MASTER_ADDR"] = "127.0.0.1"
        os.environ["MASTER_PORT"] = "29500"
        torch.distributed.init_process_group(backend="nccl")
        args.n_gpu = 1

    args.device = device
    torch.cuda.set_device(args.local_rank)
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
    )

    set_seed(args.seed)
    args_dict = vars(args)
    for k, v in args_dict.items():
        CONFIG[k] = v

    if CONFIG['temp_path'] == '':
#         if args.local_rank in [-1]:
#             os.makedirs('/test/diyi/temp', exist_ok=True)
#             temp_path = tempfile.mkdtemp(dir='/test/diyi/temp')
#         else:
#             temp_path = '/test/diyi/temp'
        CONFIG['temp_path'] = 'TEMP'
    CONFIG['emotion_vocab'] = CONFIG['temp_path'] + '/vocabs/emotion_vocab.pkl'

    if args.local_rank in [-1, 0]:
        os.makedirs(CONFIG['temp_path'], exist_ok=True)
        os.makedirs(os.path.join(CONFIG['temp_path'], 'vocabs'), exist_ok=True)
        os.makedirs(os.path.join(CONFIG['temp_path'], 'models'), exist_ok=True)
        os.makedirs(os.path.join(CONFIG['temp_path'], 'temp'), exist_ok=True)
        os.makedirs(os.path.join(CONFIG['temp_path'], 'centers'),
                    exist_ok=True)
        os.makedirs(os.path.join(CONFIG['temp_path'], 'cluster_results'),
                    exist_ok=True)


    if args.task_name == 'ERCVN':
        CONFIG['num_classes'] = no_label_data
        train_data_path = os.path.join(CONFIG['data_path'],f'{no_label_data}_label/train_data.csv')
        test_data_path = os.path.join(CONFIG['data_path'], f'{no_label_data}_label/test_data.csv')
        dev_data_path = os.path.join(CONFIG['data_path'], f'{no_label_data}_label/dev_data.csv')
        get_vocabs([train_data_path, dev_data_path, test_data_path])
        dev_dialogues = load_turn(dev_data_path)
        test_dialogues = load_turn(test_data_path)
        train_dialogues = load_turn(train_data_path)

    if CONFIG['local_rank'] != -1:
        torch.distributed.barrier()

    model = CLModel(CONFIG)

    if CONFIG['local_rank'] != -1:
        model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)

    model.to(args.device)
    if args.local_rank in [-1, 0]:
        print('---config---')
        for k, v in CONFIG.items():
            print(k, '\t\t\t', v, flush=True)

    if args.finetune:
        model, centers, centers_mask = load_latest()
    if args.train:
        train(model, train_dialogues, dev_dialogues, test_dialogues)
    if args.test:
        if args.task_name == 'ERCVN':
            test_dialogues = load_turn(test_data_path)
        model, centers, centers_mask = load_latest()
        testset = build_dataset(test_dialogues)
        best_f1, acc_diriect, acc_cluster = test(model, testset, centers, centers_mask)
        print(best_f1, acc_diriect, acc_cluster)
#     os.system('rm -r {}'.format(CONFIG['temp_path']))

get vocab from ../../DataPreprocess/Cleaned_Data/6_label/train_data.csv: 10888it [00:00, 19544.36it/s]
get vocab from ../../DataPreprocess/Cleaned_Data/6_label/dev_data.csv: 3077it [00:00, 19448.24it/s]
get vocab from ../../DataPreprocess/Cleaned_Data/6_label/test_data.csv: 1568it [00:00, 19751.59it/s]
INFO:root:total 6 emotions
processing file ../../DataPreprocess/Cleaned_Data/6_label/dev_data.csv: 3077it [00:00, 16525.78it/s]
processing file ../../DataPreprocess/Cleaned_Data/6_label/test_data.csv: 1568it [00:00, 16219.51it/s]
processing file ../../DataPreprocess/Cleaned_Data/6_label/train_data.csv: 10888it [00:00, 16261.94it/s]


---config---
bert_path 			 SIMCSE_pretrained
epochs 			 16
lr 			 0.001
ptmlr 			 1e-05
batch_size 			 32
max_len 			 256
bert_dim 			 768
pad_value 			 1
mask_value 			 64000
dropout 			 0.1
pool_size 			 256
support_set_size 			 64
num_classes 			 6
warm_up 			 128
dist_func 			 cosine
data_path 			 ../../DataPreprocess/Cleaned_Data
accumulation_steps 			 1
avg_cluster_size 			 3072
max_step 			 1024
num_positive 			 1
ratio 			 1
mu 			 0.5
cl 			 True
temperature 			 0.05
fgm 			 False
train_obj 			 spcl
speaker_vocab 			 
emotion_vocab 			 TEMP/vocabs/emotion_vocab.pkl
temp_path 			 TEMP
ngpus 			 1
device 			 cuda
CLS 			 0
SEP 			 2
test 			 True
train 			 True
finetune 			 False
print_error 			 True
output_mlp 			 False
seed 			 2333
task_name 			 ERCVN
fp_16 			 False
local_rank 			 -1
n_gpu 			 1
no_cuda 			 False


Token indices sequence length is longer than the specified maximum sequence length for this model (282 > 256). Running this sequence through the model will result in indexing errors
generate representations for all data: 100%|██| 351/351 [00:49<00:00,  7.03it/s]
INFO:root:clustering...
INFO:root:1 clusters for emotion 0
INFO:root:1 clusters for emotion 1
INFO:root:1 clusters for emotion 2
INFO:root:1 clusters for emotion 3
INFO:root:1 clusters for emotion 4
INFO:root:1 clusters for emotion 5
direct loss 0.45, instance_cl_loss 3.00: 100%|█| 173/173 [01:20<00:00,  2.15it/s
generate representations for all data: 100%|██| 349/349 [00:50<00:00,  6.97it/s]
INFO:root:clustering...
INFO:root:1 clusters for emotion 0
INFO:root:1 clusters for emotion 1
INFO:root:1 clusters for emotion 2
INFO:root:1 clusters for emotion 3
INFO:root:1 clusters for emotion 4
INFO:root:1 clusters for emotion 5
testing:  99%|█████████████████████████████████▋| 96/97 [00:13<00:00,  6.92it/s]


dev @ epoch 0 best w-f1 is 0.5893
direct f1 cls 0.5894
cluster f1 cls 0.5893


testing: 100%|██████████████████████████████████| 97/97 [00:13<00:00,  6.95it/s]
rm: cannot remove 'TEMP/models/*': No such file or directory
rm: cannot remove 'TEMP/centers/*': No such file or directory
testing: 100%|██████████████████████████████████| 49/49 [00:07<00:00,  6.90it/s]

...

best f1 test is 0.6098
acc direct test  is 0.6218
acc cluster test is 0.6020



processing file ../../DataPreprocess/Cleaned_Data/6_label/test_data.csv: 1568it [00:00, 17180.97it/s]
INFO:root:model checkpoint TEMP/models/f1_0.6285_@epoch15.pkl is loaded
INFO:root:center checkpoint TEMP/centers/f1_0.6285_@epoch15.pkl is loaded
INFO:root:centers mask checkpoint TEMP/centers/f1_0.6285_@epoch15.msk is loaded
testing: 100%|██████████████████████████████████| 49/49 [00:07<00:00,  6.93it/s]


 best w-f1 is 0.6098
direct f1 cls 0.6170
cluster f1 cls 0.6098


testing: 100%|██████████████████████████████████| 49/49 [00:07<00:00,  6.88it/s]

0.6098129605539823 0.6218112244897959 0.6020408163265306


In [12]:
# dev_dialogues

In [13]:
print(best_f1)

0.6098129605539823
